<a href="https://colab.research.google.com/github/krixik-ai/krixik-docs/blob/main/docs/examples/multi_module_non_search_pipeline_examples/multi_recursive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json
import importlib
from pathlib import Path

# preparación de demo - incuye instanciación de secretos, instalación de requerimientos, y definición de rutas
if os.getenv("COLAB_RELEASE_TAG"):
    # si estás usando este notebook en Google Colab, ingresa tus secretos acá
    MY_API_KEY = "TU_API_KEY_VA_AQUI"
    MY_API_URL = "TU_API_URL_VA_AQUI"

    # si estás usando este notebook en Google Colab, instala requerimientos y descarga los subdirectorios requeridos
    # instala el cliente Python de Krixik
    !pip install krixik

    # instala github-clone, que permite clonación fácil de los subdirectorios del repositorio de documentación XXX
    !pip install github-clone

    # clona los conjuntos de datos
    if not Path("data").is_dir():
        !ghclone XXXX #(in english it's https://github.com/krixik-ai/krixik-docs/tree/main/data)
    else:
        print("ya se clonaron los conjuntos de datos de documentación!")

    # define la variable 'data_dir' para tus rutas
    data_dir = "./data/"

    # crea directorio de salidas
    from pathlib import Path

    Path(data_dir + "/salidas").mkdir(parents=True, exist_ok=True)

    # descarga utilidades
    if not Path("utilities").is_dir():
        !ghclone XXXX # (in english it's https://github.com/krixik-ai/krixik-docs/tree/main/utilities)
    else:
        print("ya has clonado las utilidades de documentación!")
else:
    # si estás usando una descarga local de la documentación, define las rutas relativas a la estructura local de la documentación
    # importa utilidades
    sys.path.append("../../../")

    # define la variable 'data_dir' para tus rutas
    data_dir = "../../../data/"

    # si estás usando este notebook localmente desde el repositorio de documentación Krixik, carga tus secretos de un archivo .env ubicado en la base del repositorio de documentación
    from dotenv import load_dotenv

    load_dotenv("../../../.env")

    MY_API_KEY = os.getenv("MY_API_KEY")
    MY_API_URL = os.getenv("MY_API_URL")


# carga 'reset'
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline


# importa Krixik e inicializa sesión con tus secretos personales
from krixik import krixik

krixik.init(api_key=MY_API_KEY, api_url=MY_API_URL)

SUCCESS: You are now authenticated.


## *Pipeline* Multimodular: Resumen Recursivo

Una manera práctica para lograr resúmenes más cortos (tal vez más abstractos, pero igual representativos) de documentos medios o largos es resumir recursivamente. Este concepto se detalla en [la documentación](../ejemplos_pipelines_modulo_unico/unico_summarize_resumen.md) del *pipeline* de módulo único con un módulo [`summarize` (resumen)](../../modulos/modulos_ia/modulo_summarize_resumen.md), en el que se aplica ese *pipeline* varias veces para crear resúmenes más y más breves del texto de entrada.

En este documento se logra el mismo resultado con un solo *pipeline* que consiste de varios módulos [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) consecutivos. Procesar archivos a través de este *pipeline* resume recursivamente con un solo uso de *pipeline*. Si necesitas resumir aún más, puedes crear un *pipeline* similar con aún más módulos [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) en él.

El documento está dividido en las siguientes secciones:

- [Monta tu *Pipeline*](#monta-tu-pipeline)
- [Procesa un Archivo de Entrada](#procesa-un-archivo-de-entrada)

### Monta tu *Pipeline*

Para lograr lo arriba descrito, monta un pipeline que consiste de tres módulos [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) consecutivos.

Para crear el pipeline usarás el método [`create_pipeline`](../../sistema/creacion_de_pipelines/creacion_de_pipelines.md) de la siguiente manera:

In [2]:
# creación del pipeline descrito
pipeline = krixik.create_pipeline(name="multi_resumen_recursivo", module_chain=["summarize", "summarize", "summarize"])

### Procesa un Archivo de Entrada

Examina el archivo de prueba antes de continuar:

In [3]:
# examina el archivo de entrada
with open(data_dir + "input/1984_corto.txt", "r") as file:
    print(file.read())

It was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the
vile wind, slipped quickly through the glass doors of Victory Mansions,
though not quickly enough to prevent a swirl of gritty dust from entering
along with him.

The hallway smelt of boiled cabbage and old rag mats. At one end of it a
coloured poster, too large for indoor display, had been tacked to the wall.
It depicted simply an enormous face, more than a metre wide: the face of a
man of about forty-five, with a heavy black moustache and ruggedly handsome
features. Winston made for the stairs. It was no use trying the lift. Even
at the best of times it was seldom working, and at present the electric
current was cut off during daylight hours. It was part of the economy drive
in preparation for Hate Week. The flat was seven flights up, and Winston,
who was thirty-nine and had a varicose ulcer above his right ankle, went
slowly, resting se

Con el método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md) procesas esta entrada por el *pipeline*. Dado que siempre usas el modelo [predeterminado](../../modulos/modulos_ia/modulo_summarize_resumen.md#modelos-disponibles-en-el-modulo-summarize) en el módulo [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md), no hace falta usar el argumento [`modules`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md#seleccion-de-modelo-por-medio-del-argumento-modules).

In [4]:
# procesa el archivo a través del pipeline según lo arriba descrito
process_output = pipeline.process(
    local_file_path=data_dir + "input/1984_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
)

La salida del proceso se reproduce con el siguiente código. Para aprender más sobre cada componente de esta salida, revisa la documentación del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md).

El archivo de texto de salida se ha guardado en la ubicación indicada bajo `process_output_files`. El `file_id` del archivo procesado es el prefijo del nombre del archivo de salida en esta ubicación.

In [5]:
# nítidamente reproduce la salida de este proceso
print(json.dumps(process_output, indent=2))

{
  "status_code": 200,
  "pipeline": "multi_recursive_summarization",
  "request_id": "166d7e3a-ce29-4c82-9f06-17ca2016906e",
  "file_id": "33d00458-6d3d-4a43-92c4-e7027c908c38",
  "message": "SUCCESS - output fetched for file_id 33d00458-6d3d-4a43-92c4-e7027c908c38.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "../../../data/output/33d00458-6d3d-4a43-92c4-e7027c908c38.txt"
  ]
}


Para confirmar que todo salió como esperabas, carga el archivo de `process_output_files`:

In [6]:
# carga la salida del proceso del archivo
with open(process_output["process_output_files"][0], "r") as file:
    print(file.read())

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelled of boiled cabbage and old rag mats. A kilometre away, his
place of work, the Ministry of Truth, towered vast and white.


In [7]:
# elimina todos los datos procesados pertenecientes a este pipeline
reset_pipeline(pipeline)